<a href="https://colab.research.google.com/github/HMGUNASEKARA/Understanding-Keras-Tuner-Hyperparameter-Optimization-for-Neural-Networks/blob/main/Hyper_parameters_tunning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Build the simple feed Forward model

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential  # Methaninn pahala ewa input karapu nisa
from tensorflow.keras.layers import Dense       # Model eka build karanakota
from tensorflow.keras.optimizers import Adam    # tf.keras........ kiya kiya digata type karanna one na
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Example dummy data (replace with your stock dataset)
X = np.random.rand(1000, 10)  # 10 features
y = np.random.rand(1000, 1)

# Scale data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Build simple ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

history = model.fit(X_scaled, y, epochs=20, batch_size=32, validation_split=0.1)


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.2250 - val_loss: 0.1087
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1043 - val_loss: 0.1036
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0924 - val_loss: 0.1025
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0854 - val_loss: 0.1013
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0901 - val_loss: 0.1023
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0902 - val_loss: 0.1022
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0832 - val_loss: 0.1015
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0817 - val_loss: 0.1039
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0858 - val_loss: 0.1015
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0837 - val_loss: 0.1013
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0861 - val_loss: 0.1030
Epoch 12/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0815 - val_loss: 0.1

In [16]:
!pip install keras-tuner


In [17]:
import keras_tuner as kt   # Import keras tuner library and name as kt
from tensorflow.keras.layers import Dense  # Import dence layers
from tensorflow.keras.models import Sequential # Imports the Sequential model class
from tensorflow.keras.optimizers import Adam # Import adam optimizer

def build_ann(hp):  #  Define the Function , This is expect the Keras tuner,
# tuner  called this many time, Each time with different hyperparameter choises
# hp = seach space
    model = Sequential() # Create empty sqential model with object name called model

    # Here tune the number o hidden layers decide how many
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(Dense(  # Define the layers
            units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32), # Define the number of units in each layer,32 to 256
            activation=hp.Choice(f"activation_{i}", ['relu', 'tanh']) # define the activation function in each layer here used Tanh and relu.
        ))

    # Output layer
    model.add(Dense(1)) # Out put layer

    # Learning rate tuning
    lr = hp.Choice("learning_rate", [1e-4, 1e-3, 1e-2]) # Define 3 learnig rate

    model.compile(
        optimizer=Adam(learning_rate=lr), # Here used adamoptimizer with 3 learnig Rates
        loss="mse"
    )

    return model # After creating the model accoding to the specific senario return themodel


In [18]:
# Create the hyperband to tune the model
tuner = kt.Hyperband( # Create hyperband model object
    build_ann, # Function that used to build the model
    objective='val_loss',  # Tell tuner to choose the best model base on the validation loss
    max_epochs=30,  # Hyperband will train each model upto 30 epochs ba model stop early
    factor=3, # This control how aggressively reduce the the number of models
    directory='ann_tune_dir', # Give directry to the save the tunning ,Then next time this is not do this again
    project_name='ann_tuning'  # Simply look the save location
)

# Hyper band is efficent hyperparameter optimization
# Train many model with fewer epochs
# Stop bad models early
# Trains promising models fro langer
# Find the best model fast

# factor = 3  what happen here

# In the first stage hyperband train different model in lower number of epochs then remaing only
# 1/3 amout of for next stage, then the next stage remaing.. Like wise finally got the best model


Reloading Tuner from ann_tune_dir/ann_tuning/tuner0.json


In [19]:
tuner.search(X_scaled, y, epochs=30, validation_split=0.1)
# By using this code train the model in each model architectre and trail.
# Like this


In [20]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# After tuning we have different combination of parameres, Identify the best one with lowest validtion test
# Tell we need only one best posible combination

print("Best number of layers:", best_hps.get("num_layers")) # extract and print best number of layers
for i in range(best_hps.get("num_layers")): # loop through the each layer and print
    print(f"Layer {i} — Units:", best_hps.get(f"units_{i}")) # Number of layer and unit amount
    print(f"Layer {i} — Activation:", best_hps.get(f"activation_{i}")) # Number of layers and actvation

print("Best Learning Rate:", best_hps.get("learning_rate")) # Best learning rate


Best number of layers: 1
Layer 0 — Units: 32
Layer 0 — Activation: tanh
Best Learning Rate: 0.01


In [21]:
best_model = tuner.hypermodel.build(best_hps) # Tell the tuner to build the finalbest model

history = best_model.fit( # and tell train thebest model
    X_scaled, y,
    epochs=50,
    validation_split=0.1
)


Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.3798 - val_loss: 0.1081
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0923 - val_loss: 0.0988
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0849 - val_loss: 0.0987
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0861 - val_loss: 0.0990
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0862 - val_loss: 0.0982
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0850 - val_loss: 0.1159
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0903 - val_loss: 0.1019
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0822 - val_loss: 0.1036
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0837 - val_loss: 0.1034
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0883 - val_loss: 0.0992
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0834 - val_loss: 0.0987
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.08